In [ ]:
import sqlite3

import dask.dataframe as dd
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import pearsonr
from dask.distributed import Client
from matplotlib import pyplot as plt
from scipy.stats import pearsonr

from utils.constants import BigMarkets, Correspondence, SmallMarkets, MediumMarkets

In [ ]:
client = Client(n_workers=6)
client

# NBA videos dataset preparation 

In [ ]:
video_metadata_df = dd.read_parquet(f"./data/video_metadata/parquet/")
video_metadata_df["upload_date"] = dd.to_datetime(video_metadata_df.upload_date)

video_metadata_df.head(5)

In [ ]:
# Selecting only videos from the categories "Sports", "People & Blogs" and "Entertainment"

video_metadata_df_sports = video_metadata_df[video_metadata_df["categories"].isin(["Sports", "People & Blogs", "Entertainment"])]
video_metadata_df_sports["tags"] = video_metadata_df_sports["tags"].str.lower()
video_metadata_df_sports["tags"] = video_metadata_df_sports["tags"].apply(lambda x: f",{x},")

In [ ]:
# video_metadata_df_sports = video_metadata_df_sports[video_metadata_df_sports['tags'].str.contains('basketball')]

date_obj = video_metadata_df_sports.upload_date.dt
group_by = [date_obj.year, date_obj.month, video_metadata_df.categories]

video_metadata_df_sports_grouped = video_metadata_df_sports[["upload_date"]].groupby(by=group_by).count().persist()

In [ ]:
# Selecting only videos containing the tags "nba" and "basketball"

required_tags = ["nba", "basketball"]
nba_basketball_df = video_metadata_df_sports[video_metadata_df_sports["tags"].str.contains(f',{",|,".join(required_tags)},')].persist()
nba_basketball_df["year_month"] = dd.to_datetime(
    nba_basketball_df.upload_date.dt.year.astype("str") + "-" + nba_basketball_df.upload_date.dt.month.astype("str")
)

In [ ]:
nba_basketball_df.head()

# NBA teams media exposure analysis

## Media coverage of each market on Youtube

### Number of videos per team 

In [ ]:
BigMarkets

In [ ]:
MediumMarkets

In [ ]:
SmallMarkets

In [ ]:
dd_dict = {}

# Selecting the teams and their market size depending on the tags of the videos
for key, value in {**BigMarkets, **MediumMarkets, **SmallMarkets}.items():

    team_df = nba_basketball_df[nba_basketball_df["tags"].str.contains(f',{",|,".join(value)},')]
    print(key)
    team_df["team"] = key
    team_df["market_size"] = "small" if key in SmallMarkets else "medium" if key in MediumMarkets else "big"
    team_df = team_df.persist().compute()

    dd_dict[key] = team_df

In [ ]:
team_df

In [ ]:
team_df.groupby(["team"]).count()

In [ ]:
# To adapt

sns.set(rc={"figure.figsize": (11, 4)})
sns.set_style("darkgrid", {"grid.color": ".4", "grid.linestyle": ":"})
ax = plt.subplot
ax = sns.lineplot(
    x="year_month", y="win_percentage", data=lakers_df, color="r", marker="o", palette=["red"], legend="brief", label="Win Percentage Lakers"
)
ax = sns.lineplot(
    x="year_month", y="total_nba_view_percentage", data=lakers_df, marker="o", color="b", legend="brief", label="Percentage of total NBA views Lakers"
)
ax.set(xlabel="Year", ylabel="Percentage")

plt.show()  ## Number of videos per team

### Number of videos per market 

In [ ]:
team_df.groupby(["market_size"]).count()

## Number of channels per team 

In [ ]:
# groupby team and channel_id
# Select channels that have the most videos uploaded
# Select only fanbase channels - were 80% are on a single team
# vizualise the number of channels per team

## The big name bias 

### History of Succes 

### Fans engagement 